In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '112_eda_'

In [2]:
import eda

def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        
train = elo_load_data('train')
test = elo_load_data('test')
train_id = train[key].values
test_id = test[key].values
train_test = pd.concat([train, test], axis=0).reset_index(drop=True)
history = elo_load_data('historical')

100%|██████████| 3/3 [00:08<00:00,  2.87s/it]


In [6]:
auth = history.query("authorized_flag=='Y'")
hist = history.query("authorized_flag=='N'")

In [11]:
[print(col) for col in auth.columns]
pass

authorized_flag
card_id
city_id
category_1
installments
category_3
merchant_category_id
merchant_id
month_lag
purchase_amount
purchase_date
category_2
state_id
subsector_id


In [8]:
history['purchase_date'] = pd.to_datetime(history['purchase_date'])
history['elapsed_days'] = (datetime.datetime.today() - history['purchase_date']).dt.days

In [11]:
train['card_id'].shift(1).head()

0                NaN
1    C_ID_92a2005557
2    C_ID_3d0044924f
3    C_ID_d639edf6cd
4    C_ID_186d6a6901
Name: card_id, dtype: object

In [12]:
history.sort_values(by=['card_id', 'purchase_date'], inplace=True)

In [13]:
history['date_diff_1'] = history['elapsed_days'] - history.groupby(key)['elapsed_days'].shift(1)
history['date_diff_2'] = history['elapsed_days'] - history.groupby(key)['elapsed_days'].shift(2)

In [15]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [18]:
# eda_hist = train_test[['card_id', 'target', 'first_active_month']].merge(history, how='left', on=key)
eda_hist['first_active_month'] = pd.to_datetime(eda_hist['first_active_month'])
eda_hist['from_first_days'] = (eda_hist['first_active_month'] - eda_hist['purchase_date']).dt.days
print(eda_hist.shape)
eda_hist.to_csv('../eda/112_elo_eda_history_target.csv', index=True)

(29112361, 20)
